# Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# IMporting Necessary Libraries

In [ ]:
import h5py
from PIL import Image
import numpy as np, pandas as pd
import os
import io
from IPython.display import display

Extracting Images and resizign them

In [ ]:


# Define the path to your .hdf5 dataset file
hdf5_file_path = "/content/drive/MyDrive/AIS_Pro_Data/train-image.hdf5"

# Define the target image size (for example 224x224 for ResNet)
target_size = (224, 224)

In [ ]:
hdf5_file = h5py.File(hdf5_file_path, 'r')

In [ ]:
name_list = list(hdf5_file.keys()) #I'll be using this to access the images

In [ ]:
images = []

In [ ]:
# Reading image data and resizing the image so it fits resnet's required size

def image_processor(image_name):
    image_data = hdf5_file[image_name][()]
    image = Image.open(io.BytesIO(image_data))
    image_2 = image.resize(target_size)
    return image_2

In [ ]:
def image_displayer(image_name):
    image_data = hdf5_file[image_name][()]
    image = Image.open(io.BytesIO(image_data))
    display(image)

In [ ]:
images.clear()
for i in range(2):
    images.append(image_processor(name_list[i]))

Extracting labels for images

In [ ]:
train_meta = pd.read_csv("/content/drive/MyDrive/AIS_Pro_Data/train-metadata.csv")

<ipython-input-9-9acbfeca1fa1>:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_meta = pd.read_csv("/content/drive/MyDrive/AIS_Pro_Data/train-metadata.csv")


In [ ]:
train_meta.columns

Index(['isic_id', 'target', 'patient_id', 'age_approx', 'sex',
       'anatom_site_general', 'clin_size_long_diam_mm', 'image_type',
       'tbp_tile_type', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext',
       'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L',
       'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
       'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
       'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
       'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
       'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z', 'attribution', 'copyright_license', 'lesion_id',
       'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5',
       'mel_mitotic_index', '

In [ ]:
needed_column = ['isic_id', 'target']

In [ ]:
train_data = train_meta[needed_column]

In [ ]:
# Trying out on a smaller dataset

td_pos = train_data[train_data['target'] == 1]
td_neg = train_data[train_data['target'] == 0]


In [ ]:
td_pos.shape, td_neg.shape

((393, 2), (400666, 2))

In [ ]:
shuffled_td_neg = td_neg.sample(random_state=42, n=400666)

### Augmentations to make the model robust towards translations and rotations

In [ ]:
from PIL import ImageEnhance, ImageOps

def augmented_images(image):
    augmentations = []

    # Apply rotation
    augmentations.append(image.rotate(15))
    augmentations.append(image.rotate(-15))

    # Flip the image
    augmentations.append(ImageOps.mirror(image))

    # Adjust brightness
    enhancer = ImageEnhance.Brightness(image)
    augmentations.append(enhancer.enhance(1.5))
    augmentations.append(enhancer.enhance(0.7))

    # Cropping and scaling back to target size
    cropped_image = image.crop((5, 5, image.width - 5, image.height - 5)).resize(target_size)
    augmentations.append(cropped_image)

    return augmentations


In [ ]:
td_complete = pd.concat([td_pos, shuffled_td_neg])
td_complete.shape

(786, 2)

#### Retrieving all the images

In [ ]:
# Accessing images using the image names from td_complete
images.clear()
for i in range(td_complete.shape[0]):
    images.append(image_processor(td_complete['isic_id'].iloc[i]))

In [ ]:
len(images)

786

In [ ]:
# Modifying the minority images

for i in range(len(td_pos)):  # Apply to minority class only
    image = image_processor(td_pos['isic_id'].iloc[i])
   # images.append(image)  # Original image
    images.extend(augmented_images(image))  # Add augmented versions


In [ ]:
len(images)

786

In [ ]:
X = images
y = td_complete['target']

#### Train test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#### Transformations on the images to format it into model's required size, etc

In [ ]:
from torchvision import transforms
import torch

# Define transformations for your images to match ResNet input requirements
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet requires 224x224 input size
    transforms.ToTensor(),  # Convert to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

# Apply transformation to each image in your list
X_transformed = [transform(image) for image in images]

# Stack the list into a single tensor
X_tensor = torch.stack(X_transformed)


#### Setting up the dataloader

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Convert target labels into a tensor
y_tensor = torch.tensor(y.values)

# Create a dataset
dataset = TensorDataset(X_tensor, y_tensor)

# Create a DataLoader for batching
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


#### Model initialization

In [ ]:
import torch.nn as nn
from torchvision.models import resnet18

# Load ResNet18 model
model = resnet18(pretrained=True)

# Modify the final layer to output 2 classes (skin cancer - binary classification)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # For binary classification


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 71.5MB/s]


#### We are relying on crossentropy as our loss function

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()  # Loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer


#### Training model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):  # Set the number of epochs
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}")


Epoch 1, Loss: 0.8032058608531952
Epoch 2, Loss: 0.4678682720661163
Epoch 3, Loss: 0.3714549207687378
Epoch 4, Loss: 0.36422820210456847
Epoch 5, Loss: 0.2817980831861496
Epoch 6, Loss: 0.2512885445356369
Epoch 7, Loss: 0.20160607486963272
Epoch 8, Loss: 0.1759279029816389
Epoch 9, Loss: 0.2221507152915001
Epoch 10, Loss: 0.15593516692519188


In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")


Accuracy: 94.78371501272265%


In [ ]:
import joblib

#### Saving model

In [ ]:
joblib.dump(model, 'image_classifier.pkl')

['image_classifier.pkl']

In [ ]:
td_complete.columns

Index(['isic_id', 'target'], dtype='object')

In [ ]:
classifier = joblib.load('image_classifier.pkl')